In [ ]:
## set covid19_datasciente as path for python find bulletin package
import sys
from pathlib import Path
from os import getcwd, remove, chdir
from os.path import join, basename

sys.path.append(str(Path(getcwd()).parent))
current_dir = getcwd()
print(current_dir)

In [ ]:
from datetime import date

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 150)

import numpy as np
from epiweeks import Week, Year
from datetime import date, timedelta

In [ ]:
from bulletin import root, default_input, default_output, agora, hoje, ontem, anteontem, dias_apos, dias_apos_label
from bulletin.systems import Notifica
from bulletin.utils.static import Municipios
from bulletin.utils.normalize import normalize_text

In [ ]:
nt = Notifica()
nt.load(f"notifica", compress=False)

In [ ]:
for col in nt.tables['municipios'].columns:
    if col in nt.df.columns:
        del nt.df[col]

In [ ]:
# nt.df.loc[nt.df['ibge_residencia'].apply(str).str[:2]!='41', 'ibge_residencia'] = 999999
nt.df = nt.df.drop(index=nt.df.loc[nt.df['ibge_unidade_notifica'].apply(str).str[:2]!='41'].index)
# nt.df.loc[nt.df['ibge_unidade_notifica'].apply(str).str[:2]!='41', 'ibge_unidade_notifica'] = 999999

In [ ]:
nt.df.drop(index=nt.df.loc[nt.df['data_notificacao'].isna()].index, inplace=True)

In [ ]:
nt.tables['metodo']

In [ ]:
# nt.df = nt.df.loc[nt.df['metodo']==1] ## RT-PCR
# nt.df = nt.df.loc[nt.df['metodo']==2] ## Teste Rápido

In [ ]:
nt.df = nt.df.replace(nt.tables['exame'].set_index('id').to_dict())
nt.df = nt.df.replace(nt.tables['metodo'].set_index('id').to_dict())

In [ ]:
nt.df['ano'] = nt.df['data_notificacao'].dt.year
nt.df['mes'] = nt.df['data_notificacao'].dt.month
nt.df['dia'] = nt.df['data_notificacao'].dt.day

In [ ]:
nt.df.loc[nt.df['classificacao_final']==4,'classificacao_final'] = 1
nt.df.loc[nt.df['classificacao_final']==5,'classificacao_final'] = 3

In [ ]:
nt.df.groupby(['classificacao_final'])[['id']].count()#.rename(index=nt.tables['classificacao_final'].set_index('id')['classificacao_final'])

In [ ]:
dias = pd.date_range(start=date(2020,3,1), end=hoje)
ano = dias.year
mes = dias.month
dia = dias.day
dias = list(zip(ano,mes,dia))
pd.DataFrame(columns=pd.MultiIndex.from_tuples(dias))

In [ ]:
municipios = nt.tables['municipios'][['uf','macro','rs','regional','ibge','municipio','populacao']]

#populacao
municipiosPR = municipios.loc[municipios['uf'].isin(['PR'])].sort_values(['rs','ibge']).set_index(['uf','macro','rs','regional','ibge','municipio'])
rsPR = municipiosPR.groupby(['rs','regional']).sum()
macroPR = municipiosPR.groupby(['macro']).sum()
PR = municipiosPR.groupby(['uf']).sum()

In [ ]:
nt.df = nt.df.join(municipios.set_index('ibge'),on='ibge_unidade_notifica').rename(columns={'ibge_unidade_notifica':'ibge'})

In [ ]:
total_mun = nt.df.groupby(['uf','macro','rs','regional','ibge','municipio','ano','mes','dia'])[['id']].count().unstack(['ano','mes','dia']).fillna(0).astype(int).sort_index(axis=1,level=1).droplevel(axis=1,level=0)
total_mun = total_mun.reindex(municipiosPR.index,axis=0)
total_mun = total_mun.reindex(dias,axis=1).fillna(0).astype(int)

confirmados_mun = nt.df.loc[nt.df['classificacao_final']==2].groupby(['uf','macro','rs','regional','ibge','municipio','ano','mes','dia'])[['id']].count().unstack(['ano','mes','dia']).fillna(0).astype(int).sort_index(axis=1,level=1).droplevel(axis=1,level=0)
confirmados_mun = confirmados_mun.reindex(municipiosPR.index,axis=0)
confirmados_mun = confirmados_mun.reindex(dias,axis=1).fillna(0).astype(int)

positividade_mun = confirmados_mun / total_mun

In [ ]:
total_rs = total_mun.groupby(['rs','regional']).sum().fillna(0).astype(int)
confirmados_rs = confirmados_mun.groupby(['rs','regional']).sum().fillna(0).astype(int)
positividade_rs = confirmados_rs / total_rs

In [ ]:
total_macro = total_mun.groupby(['macro']).sum().fillna(0).astype(int)
confirmados_macro = confirmados_mun.groupby(['macro']).sum().fillna(0).astype(int)
positividade_macro = confirmados_macro / total_macro

In [ ]:
total = total_mun.groupby(['uf']).sum().fillna(0).astype(int)
confirmados = confirmados_mun.groupby(['uf']).sum().fillna(0).astype(int)
positividade = confirmados / total

In [ ]:
writer = pd.ExcelWriter(join(default_output,"positividade_por_dia.xlsx"))

total.to_excel(writer,'total')
confirmados.to_excel(writer,'confirmados')
positividade.to_excel(writer,'positividade')

total_macro.to_excel(writer,'total_macro')
confirmados_macro.to_excel(writer,'confirmados_macro')
positividade_macro.to_excel(writer,'positividade_macro')

total_rs.to_excel(writer,'total_rs')
confirmados_rs.to_excel(writer,'confirmados_rs')
positividade_rs.to_excel(writer,'positividade_rs')

total_mun.to_excel(writer,'total_mun')
confirmados_mun.to_excel(writer,'confirmados_mun')
positividade_mun.to_excel(writer,'positividade_mun')

writer.save()

In [ ]:
total14dias = total_mun.iloc[:,-14:].sum(axis=1).to_frame(name='total14dias')
conf14dias = confirmados_mun.iloc[:,-14:].sum(axis=1).to_frame(name='conf14dias')
posit14dias = (conf14dias / total14dias.values).rename(columns={'conf14dias':'posit14dias'})

mun = pd.concat([total14dias,conf14dias,posit14dias], axis=1)
mun.to_excel('ultimos_14_dias.xlsx')

In [ ]:
raise
path_file = join('C:\\', 'SESA', 'covid19datascience-dev', 'bulletin', 'database', 'gal')
gal = pd.read_csv(join(path_file, 'consulta-gal-complemento-2021.csv'), sep = ';')

for col in nt.tables['municipios'].columns:
    if col in gal.columns:
        if col != 'ibge':
            del gal[col]

gal['data_requisicao'] = pd.to_datetime(gal['data_requisicao'].str[:10])
gal = gal.loc[gal['ibge'].apply(str).str[:2]=='41']
gal['ibge'] = gal['ibge'].apply(str).str[:6].apply(int)
gal = gal.join(municipios.set_index('ibge'),on='ibge')

In [ ]:
# gal['unidade_requisitante','unidade_requisitante_municipio', 'unidade_requisitante_uf']

In [ ]:
# gal_por_mun = gal.loc[gal['data_requisicao'] >= pd.to_datetime(date(2022,1,1))].groupby(['uf','macro','rs','regional','ibge','municipio'])[['id']].count()
# gal_por_mun = gal_por_mun.reindex(municipiosPR.index,axis=0).fillna(0).astype(int).rename(columns={'id':'rt_pcr'})
# gal_por_mun

In [ ]:
tudo = total14dias.rename(columns={'total14dias':'tr'}).join(gal_por_mun)
tudo['total'] = tudo.sum(axis=1)
tudo

In [ ]:
tudo.reset_index().sort_values('total', ascending=False).to_excel('tr_rt_pcr_total.xlsx', index=False)